# How to match? 

## 0. Import libraries 

In [1]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.matcher import Matcher
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [2]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,0,1,521,False
1,0,0,2,529,False
2,0,0,3,501,False
3,0,0,4,465,False
4,0,0,5,461,False


In [3]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,0,0,False,525.0,423.222222
1,1,0,False,460.5,405.555556
2,2,0,False,478.0,411.777778
3,4,0,False,491.0,416.555556
4,6,0,False,489.5,411.555556
...,...,...,...,...,...
5424,9994,0,False,503.5,421.555556
5425,9995,0,False,483.0,427.777778
5426,9997,0,False,457.5,429.777778
5427,9998,0,False,511.5,422.111111


In [4]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,0,0,0,525.0,423.222222,63,1,2
1,1,0,0,460.5,405.555556,42,0,2
2,2,0,0,478.0,411.777778,23,0,2
3,4,0,0,491.0,416.555556,65,0,2
4,6,0,0,489.5,411.555556,68,0,1


In [5]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [6]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [7]:
# Standard model with base parameters
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results = model.estimate()

[25.05.2023 12:23:06 | matcher | INFO]: Applying matching
[25.05.2023 12:23:06 | Faiss matcher | INFO]: Searching index
[25.05.2023 12:23:06 | Faiss matcher | INFO]: Searching index
[25.05.2023 12:23:06 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: [10. 17. 53. 11.  7.  0.  1.  0. 10.  0.  0.  0. 10.  0.  0. 14. 13.  4.
  5. 13.  0.  0.  0.  8.  0.  4. 14. 14.  8. 15.  1.  0. 10.  1. 20.  5.
  8.  9. 10.  0. 10. 11.  5.  7.  3. 20. 26. 18. 21.  3.  6. 10.  6. 11.
  0.  7. 25.  3. 39.  3.  6.  5.  6.  4.  0. 28.  3.  5.  5.  4. 15.  0.
 61.  0.  6. 37. 25.  7.  8.  0.  1.  6.  2.  9.  2. 10. 28.  0. 15. 14.
  0. 33. 11. 13. 18. 20. 19.  5.  0. 15.  0. 34.  0.  3.  0.  7.  1. 70.
 68. 12. 18.  1. 11.  7.  0.  7. 14.  2. 20.  2.  0. 19. 21. 13.  2. 22.
  0.  3.  5.  5.  3. 18. 55. 14.  4. 24.  3.  2. 23. 11. 10. 10. 13.  5.
 21.  4.  0.  0.  2.  4. 49.  3.  0.  0.  3.  5. 48.  0.  7.  5. 18. 11.
 11.  0. 19.  6. 20. 11.  5.  9. 21.  4. 12. 1

### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [8]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [9]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)
results, quality_results = model.estimate()

TypeError: __init__() got an unexpected keyword argument 'df'

### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [ ]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)

In [ ]:
features_importance = model.lama_feature_select()
features_importance

In [ ]:
features = features_importance['Feature'].to_list()

In [ ]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results = model.estimate(features=features[:3])

In [ ]:
model.matcher.df_matched

## 3. Results  
### 3.1 ATE, ATT, ATC

In [ ]:
# model.matcher.results
results

### 3.2 SMD, PSI, KS-test, repeats

In [ ]:
# matching quality result - SMD
model.quality_result

In [ ]:
# matching quality result - PSI
model.quality_result['psi']

In [ ]:
# matching quality result - KS-test

model.quality_result['ks_test']

In [ ]:
# matching quality result - repeats
model.quality_result['repeats']

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [ ]:
 
model.validate_result()
 

In [ ]:
model.matcher.df_matched